In [16]:
import cvxpy as cp
import numpy as np
import time

In [17]:
# network structure
class TrafficNet:
    __slots__ = ('zone', 'trip', 'node', 'link', 'path', 'log', 'node_seq', 'link_seq', 'path_seq')

    def __init__(self):
        """
        Traffic network components.
        """
        self.zone = {}
        self.trip = {}
        self.node = {}
        self.link = {}
        self.path = {}
        self.log = {}  # Log for recording all the algorithmic information.
        self.node_seq = 0
        self.link_seq = 0
        self.path_seq = 0

    class Zone:

        __slots__ = ('Id', 'desList')

        def __init__(self, Id):
            self.Id = Id - 1
            self.desList = []

    class Trip:

        __slots__ = ('ori', 'des', 'dem', 'cost', 'paths', 'BB_step_size')

        def __init__(self, ori, des, dem=0.0):
            self.ori = ori
            self.des = des
            self.dem = dem
            self.cost = 0.0
            self.paths = []
            """Algorithm"""
            self.BB_step_size = 1

    class Node:
        """
        This class has attributes associated with any node
        """
        __slots__ = ('Id', 'lat', 'lon', 'outLinks', 'inLinks')

        def __init__(self, Id, lat=0.0, lon=0.0):
            self.Id = Id - 1
            self.lat = lat
            self.lon = lon
            self.outLinks = []
            self.inLinks = []

    class Link:
        """
        This class has attributes associated with any link
        """
        __slots__ = ('Id', 'tail', 'head', 'cap', 'length', 'fft', 'alpha', 'beta', 'speedLimit', 'toll', 'type',
                     'flow', 'time', 'cost', 'deri', 'integral', 'last_flow', 'Armijo_flow', 'Armijo_integral')

        def __init__(self, tail, head, cap=0.0, length=0.0, fft=0.0, alpha=0.15, beta=4, speedLimit=0.0, toll=0.0, type=1):
            """Input data"""
            self.Id = 0
            self.tail = tail
            self.head = head
            self.cap = cap
            self.length = length
            self.fft = fft
            self.alpha = alpha  # BPR function
            self.beta = beta  # BPR function
            self.speedLimit = speedLimit
            self.toll = toll
            self.type = type
            """Algorithm"""
            self.flow = 0.0
            self.time = fft  # initial setting
            self.cost = fft  # initial setting
            self.deri = 0.0
            self.integral = 0.0
            self.last_flow = 0.0
            self.Armijo_flow = 0.0
            self.Armijo_integral = 0.0

    def add_trip(self, ori, des, dem):
        """
        Add trips and zones.
        """
        if dem > 0:
            self.trip[ori, des] = self.Trip(ori, des, dem)
            if ori not in self.zone:
                self.zone[ori] = self.Zone(ori)
            if des not in self.zone:
                self.zone[des] = self.Zone(des)
            if des not in self.zone[ori].desList:
                self.zone[ori].desList.append(des)

    def add_node(self, Id, lat=0.0, lon=0.0):
        """
        Add nodes.
        """
        self.node[Id] = self.Node(Id, lat, lon)

    def add_link(self, tail, head, cap=0.0, length=0.0, fft=0.0, alpha=0.15, beta=4, speedLimit=0.0, toll=0.0, type=1):
        """
        Add links and nodes.
        """
        self.link[tail, head] = self.Link(tail, head, cap, length, fft, alpha, beta, speedLimit, toll, type)
        self.link[tail, head].Id = self.link_seq
        self.link_seq += 1
        # create node and set up node-link incidence relationship.
        if tail not in self.node:
            self.node[tail] = self.Node(tail)
        if head not in self.node:
            self.node[head] = self.Node(head)
        if (tail, head) not in self.node[tail].outLinks:
            self.node[tail].outLinks.append((tail, head))
        if (tail, head) not in self.node[head].inLinks:
            self.node[head].inLinks.append((tail, head))

In [18]:
# data
netFile = 'data\\Anaheim\\network.csv'
demFile = 'data\\Anaheim\\demand.csv'

In [19]:
# read network
'''Instantiation'''
TN = TrafficNet()

'''Read network'''
with open(netFile, 'r') as inFile:
    next(inFile)  # skip the first title line
    for line in inFile:
        # data tail,head,capacity,length,fft,alpha,beta,speedLimit,toll,link_type
        tmpIn = line.strip('\n').split(',')
        tail = int(tmpIn[0])
        head = int(tmpIn[1])
        capacity = float(tmpIn[2])
        length = float(tmpIn[3])
        fft = float(tmpIn[4])
        alpha = float(tmpIn[5])
        beta = float(tmpIn[6])
        speedLimit = float(tmpIn[7])
        toll = float(tmpIn[8])
        link_type = int(tmpIn[9])
        TN.add_link(tail, head, cap=capacity, length=length, fft=fft, alpha=alpha, beta=beta, speedLimit=speedLimit, toll=toll, type=link_type)

'''Read demand'''
with open(demFile, 'r') as inFile:
    next(inFile)  # skip the first title line
    for line in inFile:
        # data
        tmpIn = line.strip('\n').split(',')
        ori = int(tmpIn[0])
        des = int(tmpIn[1])
        demand = float(tmpIn[2])
        TN.add_trip(ori, des, demand)

In [20]:
# solve the node-link formulation of UE
x = cp.Variable(len(TN.link), nonneg=True)
xr = cp.Variable((len(TN.node), len(TN.link)), nonneg=True)

tmp = 0
for l in TN.link:
    tmp += TN.link[l].fft * x[TN.link[l].Id] + TN.link[l].fft * TN.link[l].alpha * TN.link[l].cap / (
        TN.link[l].beta + 1) * (x[TN.link[l].Id] / TN.link[l].cap) ** (TN.link[l].beta + 1)
obj_func = cp.Minimize(tmp)

constraints = []
for r in TN.zone:
    for j in TN.node:
        if j == r:
            qrj = -sum(TN.trip[r, s].dem for s in TN.zone[r].desList)
        elif j in TN.zone[r].desList:
            qrj = TN.trip[r, j].dem
        else:
            qrj = 0
        constraints += [sum(xr[TN.zone[r].Id, TN.link[l].Id] for l in TN.node[j].inLinks) 
                        - sum(xr[TN.zone[r].Id, TN.link[l].Id] for l in TN.node[j].outLinks) == qrj]

for l in TN.link:
    constraints += [x[TN.link[l].Id] == sum(xr[TN.zone[r].Id, TN.link[l].Id] for r in TN.zone)]

problem = cp.Problem(obj_func, constraints)

start = time.time()
problem.solve(verbose=True, abstol=1e-4)
end = time.time()
print('Solving time:', end - start)

d:\Software\Anaconda\Lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) May 21 04:43:00 PM: Your problem has 381138 variables, 16722 constraints, and 0 parameters.
(CVXPY) May 21 04:43:02 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 21 04:43:02 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 21 04:43:02 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 21 04:43:02 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 21 04:43:04 PM: Compiling problem (target solver=ECO

In [ ]:
for ind, l in enumerate(TN.link):
        print(f"link {l}, flow = {x.value[ind]}")

link (1, 2), flow = 4494.630768249446
link (1, 3), flow = 8119.061469974866
link (2, 1), flow = 4519.061599895555
link (2, 6), flow = 5967.3460909078785
link (3, 1), flow = 8094.630638329298
link (3, 4), flow = 14006.37348088574
link (3, 12), flow = 10022.333058111462
link (4, 3), flow = 14030.557688876403
link (4, 5), flow = 18006.372832511515
link (4, 11), flow = 5200.0056763445045
link (5, 4), flow = 18030.5585398876
link (5, 6), flow = 8798.27206914344
link (5, 9), flow = 15780.810888529812
link (6, 2), flow = 5991.776922554288
link (6, 5), flow = 8806.506435299967
link (6, 8), flow = 12492.90280772772
link (7, 8), flow = 12101.5185153113
link (7, 18), flow = 15794.279427278163
link (8, 6), flow = 12525.568005530795
link (8, 7), flow = 12040.873642778683
link (8, 9), flow = 6882.669274546405
link (8, 16), flow = 8388.71857102104
link (9, 5), flow = 15796.76222974983
link (9, 8), flow = 6836.688580322688
link (9, 10), flow = 21744.12282675799
link (10, 9), flow = 21814.09347375456
l

The CVXPY solver cannot get a solution for the UE problem on the Winnipeg network within 2 hours.